In [1]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
import copy

In [2]:
es = Elasticsearch([
    'localhost',
    '***REMOVED***'
], verify_certs=True)

In [3]:
es_local = Elasticsearch('localhost')
es_aws = Elasticsearch('***REMOVED***', verify_certs=True)

In [4]:
es_aws.indices.get_aliases().keys()

[u'comparatory']

In [5]:
es_aws.indices.get_field_mapping('dets.COMPANY CONFORMED NAME', 'comparatory', 'company')

{u'comparatory': {u'mappings': {u'company': {u'dets.COMPANY CONFORMED NAME': {u'full_name': u'dets.COMPANY CONFORMED NAME',
     u'mapping': {u'COMPANY CONFORMED NAME': {u'analyzer': u'autocomplete',
       u'search_analyzer': u'standard',
       u'type': u'string'}}}}}}}

In [6]:
es_aws.count('comparatory')

{u'_shards': {u'failed': 0, u'successful': 3, u'total': 3}, u'count': 0}

### Get all docs from MongoDB

In [7]:
client = MongoClient()
db = client.industry
ten_k = db.dataset

In [8]:
docs_orig = list(ten_k.find())

### Doc-by-doc upload to AWS elasticseach node
AWS has batch limit of 10MB

In [9]:
docs = copy.deepcopy(docs_orig)
print len(docs)

7842


In [10]:
for d in docs:
    _id = d.pop('_id')
    es_local.index(index='comparatory', doc_type='company', body=d, id=_id)
    es_aws.index(index='comparatory', doc_type='company', body=d, id=_id)